**Group 11**

In [1]:
from scipy.optimize import linprog
import numpy as np

In [2]:
def branch_and_bound(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None):

    def solve_relaxed(c, A_ub, b_ub, A_eq, b_eq, bounds):
        result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
        if result.success:
            return result.x, result.fun
        else:
            return None, None

    def is_integer(x):
        return np.allclose(x, np.round(x))

    best_solution = None
    best_obj_value = float('inf')
    stack = [(bounds)]

    while stack:
        current_bounds = stack.pop()

        solution, obj_value = solve_relaxed(c, A_ub, b_ub, A_eq, b_eq, current_bounds)
        print(solution)

        if solution is None:
            continue

        if is_integer(solution):
            if obj_value < best_obj_value:
                best_solution = solution
                best_obj_value = obj_value
                break

        else:
            for i in range(len(solution)):
                if not np.isclose(solution[i], np.round(solution[i])):
                    lower_bounds = current_bounds.copy()
                    upper_bounds = current_bounds.copy()

                    lower_bounds[i] = (current_bounds[i][0], np.floor(solution[i]))
                    upper_bounds[i] = (np.ceil(solution[i]), current_bounds[i][1])

                    stack.append(lower_bounds)
                    stack.append(upper_bounds)
                    break

    return np.round(best_solution), best_obj_value

In [3]:
# Example
c = [4, 3, 5]  # Coefficients for the objective function (minimze) z = c.T*x
A = [[-2,-3,-1],[-1,-2,-4],[-3,-1,-2],[1,0,1],[0,1,2]]  # Coefficient matrix for inequalities Ax <= b
b = [-10,-15,-12,7,6]  # Right-hand side of inequalities (Ax <= b)
A_eq = [[0,0,0]] # Coefficient matrix of inequalities (Ax = b)
b_eq = [0] # Right-hand side of inequalities (Ax = b)
bounds = [(0, None), (0, None), (0, None)]  # Variable bounds (x, y >= 0)

# Solve the integer programming problem using Branch and Bound
solution, objective_value = branch_and_bound(c, A, b, A_eq, b_eq,bounds)
print(f"Optimal integer solution: {solution}")
print(f"Objective value: {objective_value}")

[3.  0.4 2.8]
[3.  1.  2.5]
None
[3. 2. 2.]
Optimal integer solution: [3. 2. 2.]
Objective value: 28.0
